# Setup and Context

### Introduction

On November 27, 1895, Alfred Nobel signed his last will in Paris. When it was opened after his death, the will caused a lot of controversy, as Nobel had left much of his wealth for the establishment of a prize.

Alfred Nobel dictates that his entire remaining estate should be used to endow “prizes to those who, during the preceding year, have conferred the greatest benefit to humankind”.

Every year the Nobel Prize is given to scientists and scholars in the categories chemistry, literature, physics, physiology or medicine, economics, and peace. 

<br>

<img src=https://i.imgur.com/36pCx5Q.jpg>

Let's see what patterns we can find in the data of the past Nobel laureates. What can we learn about the Nobel prize and our world more generally?

### Upgrade plotly (only Google Colab Notebook)

Google Colab may not be running the latest version of plotly. If you're working in Google Colab, uncomment the line below, run the cell, and restart your notebook server. 

In [ ]:
# %pip install --upgrade plotly

### Import Statements

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

### Notebook Presentation

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

### Read the Data

In [ ]:
df_data = pd.read_csv('nobel_prize_data.csv')

Caveats: The exact birth dates for Michael Houghton, Venkatraman Ramakrishnan, and Nadia Murad are unknown. I've substituted them with mid-year estimate of July 2nd. 


In [ ]:
df_data.head()

In [ ]:
df_data.info()

# Data Exploration & Cleaning

**Challenge**: Preliminary data exploration. 
* What is the shape of `df_data`? How many rows and columns?
* What are the column names?
* In which year was the Nobel prize first awarded?
* Which year is the latest year included in the dataset?

In [ ]:
print(f"DataFrame shape is: {df_data.shape}, it has {df_data.shape[0]} rows and {df_data.shape[1]} columns.")

In [ ]:
print(f"Column names:\n{df_data.columns}")
print(f"Nobel prize was first awarded at {df_data['year'].min()}")
print(f"Latest nobel prize data in this dataset is at {df_data['year'].max()}")

**Challange**: 
* Are there any duplicate values in the dataset?
* Are there NaN values in the dataset?
* Which columns tend to have NaN values?
* How many NaN values are there per column? 
* Why do these columns have NaN values?  

### Check for Duplicates

In [ ]:
df_data.duplicated().values.any()  # no duplicates

### Check for NaN Values

In [ ]:
df_data.isna().values.any()  # There is NaN Values

In [ ]:
nan_columns = df_data.isna().any()
nan_columns

In [ ]:
# NaN values per column
for column in nan_columns.index:
    print(f"{column} column has {df_data[column].isna().sum()} NaN value(s).")

#### Why do these columns have NaN values?
* Awarded people may not have joined an organization.
* The ISO codes for their countries may not be available.
* Issues in birth certificates, some may have been left his country by certain conditions

In [ ]:
col_subset = ['year', 'category', 'laureate_type',
              'birth_date', 'full_name', 'organization_name']
df_data.loc[df_data.birth_date.isna()][col_subset]

In [ ]:
nan_rows = df_data.isna().any(axis=1)
print(f"We have {nan_rows.sum()} rows with NaN values.")
df_data[nan_rows]

### Type Conversions

**Challenge**: 
* Convert the `birth_date` column to Pandas `Datetime` object.
* Add a Column called `share_pct` which has the laureates' share as a percentage in the form of a floating-point number.

#### Birth Date to Datetime

In [ ]:
df_data["birth_date"] = pd.to_datetime(df_data["birth_date"])
df_data.info()

#### Add a Column with the Prize Share as a Percentage

#### My Solution

In [ ]:
prize_share_values = df_data.prize_share.astype(str).str.split("/").apply(lambda x: [int(x[0]), int(x[1])])
prize_share_values = np.array(prize_share_values.tolist())
win_from = prize_share_values[:, 0]
total_winners = prize_share_values[:, 1]
df_data["share_pct"] = win_from / total_winners
df_data

In [ ]:
df_data.info()

#### Angela's Solution

In [ ]:
df_data_angela = df_data
separated_values = df_data_angela.prize_share.str.split('/', expand=True)
numerator = pd.to_numeric(separated_values[0])
denomenator = pd.to_numeric(separated_values[1])
df_data_angela['share_pct'] = numerator / denomenator
df_data_angela.head()

# Plotly Donut Chart: Percentage of Male vs. Female Laureates

**Challenge**: Create a [donut chart using plotly](https://plotly.com/python/pie-charts/) which shows how many prizes went to men compared to how many prizes went to women. What percentage of all the prizes went to women?

In [ ]:
prizes_by_gender = df_data["sex"].value_counts()
prizes_by_gender

In [ ]:
pie_figure = px.pie(
    labels=prizes_by_gender.index,
    values=prizes_by_gender.values,
    color=prizes_by_gender.index,
    names=prizes_by_gender.index,
    title="Men vs Women prize win",
    hole=0.4
)

pie_figure.update_traces(
    textposition="outside",
    textinfo="percent+label"
)

pie_figure.show()

# Who were the first 3 Women to Win the Nobel Prize?

**Challenge**: 
* What are the names of the first 3 female Nobel laureates? 
* What did the win the prize for? 
* What do you see in their `birth_country`? Were they part of an organisation?

In [ ]:
# The first three women to win Nobel Prize.
first3_women_df = df_data[df_data["sex"] == "Female"].sort_values(by="year", ascending=True).head(3)
first3_women_df

In [ ]:
# Prizes they won.
first3_women_df[["year", "full_name", "category", "prize"]]

In [ ]:
# They were not member of any organization
first3_women_df[["birth_country", "organization_name", "organization_country", "organization_city"]]

# Find the Repeat Winners

**Challenge**: Did some people get a Nobel Prize more than once? If so, who were they? 

#### My solution

In [ ]:
# Who won the prize more than once?
repeated_winners = df_data["full_name"].value_counts()

In [ ]:
mask = repeated_winners.values > 1
repeated_winners[mask]

#### Angela's Solution (Better)

In [ ]:
is_winner = df_data.duplicated(subset=['full_name'], keep=False)
multiple_winners = df_data[is_winner]
print(f'There are {multiple_winners.full_name.nunique()}'
      ' winners who were awarded the prize more than once.')


In [ ]:
col_subset = ['year', 'category', 'laureate_type', 'full_name']
multiple_winners[col_subset]

# Number of Prizes per Category

**Challenge**: 
* In how many categories are prizes awarded? 
* Create a plotly bar chart with the number of prizes awarded by category. 
* Use the color scale called `Aggrnyl` to color the chart, but don't show a color axis.
* Which category has the most number of prizes awarded? 
* Which category has the lowest number of prizes awarded? 

In [ ]:
# how many categories are prizes awarded?
print(f"There are {df_data['category'].nunique()} different categories where Nobel Prize is awarded")

In [ ]:
awards_by_category = df_data["category"].value_counts()
awards_by_category

In [ ]:
# Plotly bar chart with the number of prizes awarded by category
awards_bar = px.bar(
    awards_by_category,
    x=awards_by_category.index,
    y=awards_by_category.values,
    title="Number of prizes won for each category",
    color=awards_by_category.values,
    color_continuous_scale="Aggrnyl"
)

awards_bar.update_layout(
    xaxis_title="Category",
    yaxis_title="Prizes Won",
    coloraxis_showscale=False
)

awards_bar.show()

In [ ]:
# Medicine has the most number of prizes, while the Economics is the lowest.

**Challenge**: 
* When was the first prize in the field of Economics awarded?
* Who did the prize go to?

In [ ]:
first_to_win_economics_prize = df_data.loc[df_data["category"] == "Economics"]
first_to_win_economics_prize = first_to_win_economics_prize.sort_values(by="year", ascending=True)
first_to_win_economics_prize["full_name"].head(1)

# Male and Female Winners by Category

**Challenge**: Create a [plotly bar chart](https://plotly.com/python/bar-charts/) that shows the split between men and women by category. 
* Hover over the bar chart. How many prizes went to women in Literature compared to Physics?

<img src=https://i.imgur.com/od8TfOp.png width=650>

In [ ]:
cat_men_women = df_data.groupby(['category', 'sex'],
                                as_index=False).agg({'prize': pd.Series.count})
cat_men_women.sort_values('prize', ascending=False, inplace=True)
cat_men_women

In [ ]:
cat_men_women_bar = px.bar(
    cat_men_women,
    x="category",
    y="prize",
    title="Number of Prizes Awarded per Category split by Men and Women",
    color="sex"
)

cat_men_women_bar.show()

# Number of Prizes Awarded Over Time

**Challenge**: Are more prizes awarded recently than when the prize was first created? Show the trend in awards visually. 
* Count the number of prizes awarded every year. 
* Create a 5 year rolling average of the number of prizes (Hint: see previous lessons analysing Google Trends).
* Using Matplotlib superimpose the rolling average on a scatter plot.
* Show a tick mark on the x-axis for every 5 years from 1900 to 2020. (Hint: you'll need to use NumPy). 

<img src=https://i.imgur.com/4jqYuWC.png width=650>

* Use the [named colours](https://matplotlib.org/3.1.0/gallery/color/named_colors.html) to draw the data points in `dogerblue` while the rolling average is coloured in `crimson`. 

<img src=https://i.imgur.com/u3RlcJn.png width=350>

* Looking at the chart, did the first and second world wars have an impact on the number of prizes being given out? 
* What could be the reason for the trend in the chart?


In [ ]:
year_prize_count = df_data.groupby("year")["prize"].count()
year_prize_count

In [ ]:
year_prize_rolling_avg = year_prize_count.rolling(window=5).mean()

In [ ]:
# Angel solution (can't solve it on my own)
plt.figure(figsize=(16,8), dpi=200)
plt.title('Number of Nobel Prizes Awarded per Year', fontsize=18)
plt.yticks(fontsize=14)
plt.xticks(ticks=np.arange(1900, 2021, step=5), 
           fontsize=14, 
           rotation=45)
 
ax = plt.gca() # get current axis
ax.set_xlim(1900, 2020)
 
ax.scatter(x=year_prize_count.index, 
           y=year_prize_count.values, 
           c='dodgerblue',
           alpha=0.7,
           s=100,)
 
ax.plot(year_prize_count.index, 
        year_prize_rolling_avg.values, 
        c='crimson', 
        linewidth=3,)
 
plt.show()

# The Countries with the Most Nobel Prizes

**Challenge**: 
* Create a Pandas DataFrame called `top20_countries` that has the two columns. The `prize` column should contain the total number of prizes won. 

<img src=https://i.imgur.com/6HM8rfB.png width=350>

* Is it best to use `birth_country`, `birth_country_current` or `organization_country`? 
* What are some potential problems when using `birth_country` or any of the others? Which column is the least problematic? 
* Then use plotly to create a horizontal bar chart showing the number of prizes won by each country. Here's what you're after:

<img src=https://i.imgur.com/agcJdRS.png width=750>

* What is the ranking for the top 20 countries in terms of the number of prizes?

In [ ]:
top20_countries = df_data.groupby("birth_country_current", as_index=False)["prize"].count()
top20_countries.sort_values(by="prize", inplace=True)
top20_countries = top20_countries[-20:]
top20_countries

In [ ]:
horizontal_countries_bar = px.bar(
    top20_countries,
    x="prize",
    y="birth_country_current",
    title="Top 20 Countries by Number of Prizes",
    orientation='h',
    color="prize",
)

horizontal_countries_bar.update_layout(
    xaxis_title = "Number of Prizes",
    yaxis_title = "Country",
    coloraxis_showscale=False,
)

horizontal_countries_bar.show()

# Use a Choropleth Map to Show the Number of Prizes Won by Country

* Create this choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/):

<img src=https://i.imgur.com/s4lqYZH.png>

* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 

Hint: You'll need to use a 3 letter country code for each country. 


# In Which Categories are the Different Countries Winning Prizes? 

**Challenge**: See if you can divide up the plotly bar chart you created above to show the which categories made up the total number of prizes. Here's what you're aiming for:

<img src=https://i.imgur.com/iGaIKCL.png>

* In which category are Germany and Japan the weakest compared to the United States?
* In which category does Germany have more prizes than the UK?
* In which categories does France have more prizes than Germany?
* Which category makes up most of Australia's nobel prizes?
* Which category makes up half of the prizes in the Netherlands?
* Does the United States have more prizes in Economics than all of France? What about in Physics or Medicine?


The hard part is preparing the data for this chart! 


*Hint*: Take a two-step approach. The first step is grouping the data by country and category. Then you can create a DataFrame that looks something like this:

<img src=https://i.imgur.com/VKjzKa1.png width=450>


### Number of Prizes Won by Each Country Over Time

* When did the United States eclipse every other country in terms of the number of prizes won? 
* Which country or countries were leading previously?
* Calculate the cumulative number of prizes won by each country in every year. Again, use the `birth_country_current` of the winner to calculate this. 
* Create a [plotly line chart](https://plotly.com/python/line-charts/) where each country is a coloured line. 

# What are the Top Research Organisations?

**Challenge**: Create a bar chart showing the organisations affiliated with the Nobel laureates. It should looks something like this:

<img src=https://i.imgur.com/zZihj2p.png width=600>

* Which organisations make up the top 20?
* How many Nobel prize winners are affiliated with the University of Chicago and Harvard University?

# Which Cities Make the Most Discoveries? 

Where do major discoveries take place?  

**Challenge**: 
* Create another plotly bar chart graphing the top 20 organisation cities of the research institutions associated with a Nobel laureate. 
* Where is the number one hotspot for discoveries in the world?
* Which city in Europe has had the most discoveries?

# Where are Nobel Laureates Born? Chart the Laureate Birth Cities 

**Challenge**: 
* Create a plotly bar chart graphing the top 20 birth cities of Nobel laureates. 
* Use a named colour scale called `Plasma` for the chart.
* What percentage of the United States prizes came from Nobel laureates born in New York? 
* How many Nobel laureates were born in London, Paris and Vienna? 
* Out of the top 5 cities, how many are in the United States?


# Plotly Sunburst Chart: Combine Country, City, and Organisation

**Challenge**: 

* Create a DataFrame that groups the number of prizes by organisation. 
* Then use the [plotly documentation to create a sunburst chart](https://plotly.com/python/sunburst-charts/)
* Click around in your chart, what do you notice about Germany and France? 


Here's what you're aiming for:

<img src=https://i.imgur.com/cemX4m5.png width=300>



# Patterns in the Laureate Age at the Time of the Award

How Old Are the Laureates When the Win the Prize?

**Challenge**: Calculate the age of the laureate in the year of the ceremony and add this as a column called `winning_age` to the `df_data` DataFrame. Hint: you can use [this](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to help you. 



### Who were the oldest and youngest winners?

**Challenge**: 
* What are the names of the youngest and oldest Nobel laureate? 
* What did they win the prize for?
* What is the average age of a winner?
* 75% of laureates are younger than what age when they receive the prize?
* Use Seaborn to [create histogram](https://seaborn.pydata.org/generated/seaborn.histplot.html) to visualise the distribution of laureate age at the time of winning. Experiment with the number of `bins` to see how the visualisation changes.

### Descriptive Statistics for the Laureate Age at Time of Award

* Calculate the descriptive statistics for the age at the time of the award. 
* Then visualise the distribution in the form of a histogram using [Seaborn's .histplot() function](https://seaborn.pydata.org/generated/seaborn.histplot.html).
* Experiment with the `bin` size. Try 10, 20, 30, and 50.  

### Age at Time of Award throughout History

Are Nobel laureates being nominated later in life than before? Have the ages of laureates at the time of the award increased or decreased over time?

**Challenge**

* Use Seaborn to [create a .regplot](https://seaborn.pydata.org/generated/seaborn.regplot.html?highlight=regplot#seaborn.regplot) with a trendline.
* Set the `lowess` parameter to `True` to show a moving average of the linear fit.
* According to the best fit line, how old were Nobel laureates in the years 1900-1940 when they were awarded the prize?
* According to the best fit line, what age would it predict for a Nobel laureate in 2020?


### Winning Age Across the Nobel Prize Categories

How does the age of laureates vary by category? 

* Use Seaborn's [`.boxplot()`](https://seaborn.pydata.org/generated/seaborn.boxplot.html?highlight=boxplot#seaborn.boxplot) to show how the mean, quartiles, max, and minimum values vary across categories. Which category has the longest "whiskers"? 
* In which prize category are the average winners the oldest?
* In which prize category are the average winners the youngest?

**Challenge**
* Now use Seaborn's [`.lmplot()`](https://seaborn.pydata.org/generated/seaborn.lmplot.html?highlight=lmplot#seaborn.lmplot) and the `row` parameter to create 6 separate charts for each prize category. Again set `lowess` to `True`.
* What are the winning age trends in each category? 
* Which category has the age trending up and which category has the age trending down? 
* Is this `.lmplot()` telling a different story from the `.boxplot()`?
* Create another chart with Seaborn. This time use `.lmplot()` to put all 6 categories on the same chart using the `hue` parameter. 
